## Hypersonalization of music recommendation taking into account weather dynamics


In [ ]:
pip install -U pywttr pywttr-models

In [ ]:
import pandas as pd
import numpy as np

user_df = pd.read_csv('user1.csv')
user_df.head()

,Unnamed: 0,track_name,artist_name
0,0,Never Let You Go,Justin Bieber
1,1,House Of Balloons / Glass Table Girls - Original,The Weeknd
2,2,Solo,Ludovico Einaudi
3,3,NASA,Ariana Grande
4,4,Chanson D’Amour,Ludovico Einaudi


##Function to start recommendation process
- Enter city
- Enter WWS
- Path to playlist with track and artist_name
- Path to DB (1.2M tracks)
- get you recommendations
- wait till i create a playlist manually for you


In [ ]:
if __name__ == "__main__":
    import pywttr
    import pandas as pd
    import numpy as np
    import pywttr
    from tkinter import Tk, filedialog

    def categorize_weather(description):
        clusters = {
            "Sunny": ["Clear", "Sunny"],
            "Cloudy": [
                "Partly cloudy", "Cloudy", "Overcast", "Mist", "Haze", "Fog", "Smoke",
                "Partial fog", "Shallow fog", "Patchy fog possible", "Freezing fog"
            ],
            "Rainy": [
                "Drizzle", "Light drizzle", "Patchy light drizzle", "Heavy freezing drizzle",
                "Freezing drizzle", "Rain shower", "Patchy rain possible", "Patchy light rain",
                "Light rain", "Moderate rain at times", "Moderate rain", "Heavy rain at times",
                "Heavy rain", "Light freezing rain", "Moderate or heavy freezing rain", "Light sleet",
                "Moderate or heavy sleet", "Ice pellets", "Light rain shower", "Moderate or heavy rain shower",
                "Torrential rain shower", "Light sleet showers", "Moderate or heavy sleet showers",
                "Patchy light rain with thunder", "Moderate or heavy rain with thunder",
                "Rain and snow shower", "Rain with thunderstorm", "Heavy rain with thunderstorm",
                "Rain and hail with thunderstorm"
            ],
            "Snowy": [
                "Snow", "Light snow", "Heavy snow", "Snow shower", "Light snow shower", "Heavy snow shower",
                "Patchy snow possible", "Patchy light snow", "Patchy moderate snow", "Moderate snow",
                "Patchy heavy snow", "Blizzard", "Blowing snow", "Light rain and snow",
                "Moderate or heavy snow showers", "Moderate or heavy snow with thunder",
                "Patchy light snow with thunder", "Snow and hail with thunderstorm"
            ]
        }

        for category, descriptions in clusters.items():
            if description in descriptions:
                return category
        return "Unknown"

    # Function to calculate weather score
    def calculate_weather_score(song, weather_rules):
        acousticness_score = weather_rules['acousticness_range'][0] <= song['acousticness'] <= weather_rules['acousticness_range'][1]
        tempo_score = weather_rules['tempo_range'][0] <= song.get('tempo', 0) <= weather_rules['tempo_range'][1]
        energy_score = weather_rules['energy_range'][0] <= song['energy'] <= weather_rules['energy_range'][1]
        valence_score = weather_rules['valence_range'][0] <= song.get('valence', 0) <= weather_rules['valence_range'][1]
        return (acousticness_score + tempo_score + energy_score + valence_score) / 4

    # Function to calculate similarity score
    def calculate_similarity(song, user_playlist):
        user_features = user_playlist[['acousticness', 'energy', 'valence', 'tempo', 'danceability']].mean().to_numpy()
        song_features = song[['acousticness', 'energy', 'valence', 'tempo', 'danceability']].to_numpy()
        similarity = 1 - np.linalg.norm(user_features - song_features)
        user_genres = user_playlist['genre'].value_counts(normalize=True).to_dict()
        genre_similarity = user_genres.get(song['genre'], 0)
        return 0.7 * similarity + 0.3 * genre_similarity
    weather_mapping = {
    ('rainy', 'spring'): {'acousticness_range': (0.7, 1.0), 'tempo_range': (0, 90), 'energy_range': (0.3, 0.7), 'valence_range': (0.3, 0.7)},
    ('rainy', 'summer'): {'acousticness_range': (0.7, 1.0), 'tempo_range': (0, 90), 'energy_range': (0.0, 0.3), 'valence_range': (0.3, 0.7)},
    ('rainy', 'autumn'): {'acousticness_range': (0.7, 1.0), 'tempo_range': (0, 90), 'energy_range': (0.0, 0.3), 'valence_range': (0.0, 0.3)},
    ('rainy', 'winter'): {'acousticness_range': (0.7, 1.0), 'tempo_range': (90, 120), 'energy_range': (0.0, 0.3), 'valence_range': (0.0, 0.3)},
    ('snowy', 'spring'): {'acousticness_range': (0.7, 1.0), 'tempo_range': (90, 120), 'energy_range': (0.0, 0.3), 'valence_range': (0.3, 0.7)},
    ('snowy', 'summer'): {'acousticness_range': (0.7, 1.0), 'tempo_range': (0, 90), 'energy_range': (0.0, 0.3), 'valence_range': (0.0, 0.3)},
    ('snowy', 'autumn'): {'acousticness_range': (0.7, 1.0), 'tempo_range': (0, 90), 'energy_range': (0.0, 0.3), 'valence_range': (0.3, 0.7)},
    ('snowy', 'winter'): {'acousticness_range': (0.7, 1.0), 'tempo_range': (90, 120), 'energy_range': (0.3, 0.7), 'valence_range': (0.7, 1.0)},
    ('sunny', 'spring'): {'acousticness_range': (0.3, 0.7), 'tempo_range': (90, 120), 'energy_range': (0.3, 0.7), 'valence_range': (0.7, 1.0)},
    ('sunny', 'summer'): {'acousticness_range': (0.0, 0.3), 'tempo_range': (120, 180), 'energy_range': (0.7, 1.0), 'valence_range': (0.7, 1.0)},
    ('sunny', 'autumn'): {'acousticness_range': (0.3, 0.7), 'tempo_range': (90, 120), 'energy_range': (0.3, 0.7), 'valence_range': (0.3, 0.7)},
    ('sunny', 'winter'): {'acousticness_range': (0.3, 0.7), 'tempo_range': (90, 120), 'energy_range': (0.7, 1.0), 'valence_range': (0.7, 1.0)},
    ('cloudy', 'spring'): {'acousticness_range': (0.7, 1.0), 'tempo_range': (90, 120), 'energy_range': (0.0, 0.3), 'valence_range': (0.3, 0.7)},
    ('cloudy', 'summer'): {'acousticness_range': (0.7, 1.0), 'tempo_range': (90, 120), 'energy_range': (0.0, 0.3), 'valence_range': (0.3, 0.7)},
    ('cloudy', 'autumn'): {'acousticness_range': (0.7, 1.0), 'tempo_range': (0, 90), 'energy_range': (0.0, 0.3), 'valence_range': (0.0, 0.3)},
    ('cloudy', 'winter'): {'acousticness_range': (0.7, 1.0), 'tempo_range': (0, 90), 'energy_range': (0.3, 0.7), 'valence_range': (0.3, 0.7)}
  }
    # Step 1: Get user inputs
    city = input("Enter the city for weather data: ")
    WSS = float(input("Enter the Weather Sensitivity Score (0 to 1): "))

    # File input for user songs
    file_path = input("Enter the path to the file containing the songs (CSV with track_name and artist_name): ")
    try:
        user_df = pd.read_csv(file_path)
    except Exception as e:
        print(f"Error reading user file: {e}")
        exit()

    # Validate file columns
    if not {'track_name', 'artist_name'}.issubset(user_df.columns):
        print("The file must contain 'track_name' and 'artist_name' columns.")
        exit()

    # Load result_df with music characteristics
    try:
        result_df_path = input("Enter the path to the dataset with music characteristics (e.g., result_df.csv): ")
        result_df = pd.read_csv(result_df_path)
        print("Music characteristics dataset loaded successfully.")
    except Exception as e:
        print(f"Error loading music characteristics dataset: {e}")
        exit()

    # Step 2: Retrieve weather data
    language = pywttr.Language.RU
    with pywttr.Wttr() as wttr:
        weather = wttr.weather(city, language=language)

    current_conditions = weather.current_condition[0]
    weather_desc = current_conditions.weather_desc[0].value
    season = "winter"  # This can be dynamically set based on the date or user input
    categorized_weather = categorize_weather(weather_desc)

    print(f"City: {city}, Weather: {weather_desc}, Category: {categorized_weather}")

    # Step 3: Merge user data with dataset
    try:
        sampled_songs = result_df.merge(user_df, on=['track_name', 'artist_name'], how='inner')
    except Exception as e:
        print(f"Error during merge: {e}")
        exit()

    if sampled_songs.empty:
        print("No matching songs found in the dataset.")
        exit()

    # Weather rules
    current_weather = (categorized_weather.lower(), season)
    weather_rules = weather_mapping[current_weather]

    # Step 4: Calculate scores
    result_df['weather_score'] = result_df.apply(lambda song: calculate_weather_score(song, weather_rules), axis=1)
    result_df['similarity_score'] = result_df.apply(lambda song: calculate_similarity(song, sampled_songs), axis=1)

    # Step 3: Combine Scores with WSS Influence
    result_df['final_score'] = (
        np.minimum(WSS, 0.5) * result_df['weather_score'] +
        np.maximum(1 - WSS, 0.5) * result_df['similarity_score']
    )

    # Step 5: Recommend top 15 songs
    recommendations = result_df.sort_values('final_score', ascending=False).head(15)
    print(recommendations[['track_name', 'artist_name', 'final_score', 'similarity_score', 'weather_score', 'genre']])


Enter the city for weather data: Санкт-Петербург
Enter the Weather Sensitivity Score (0 to 1): 0.75
Enter the path to the file containing the songs (CSV with track_name and artist_name): user1.csv
Enter the path to the dataset with music characteristics (e.g., result_df.csv): database.csv
Music characteristics dataset loaded successfully.
City: Санкт-Петербург, Weather: Light freezing rain, Category: Rainy
                                                track_name  \
883452   Concerto for Harpsichord and Orchestra: Moveme...   
800270                                           Bluebeard   
882683                                  Death Is a Disease   
47862                         What Was Good Enough for You   
928612                                  Salvation Mountain   
921528                   Till You Lay Down Your Heavy Load   
1160558                             Son of Placenta Previa   
795515            Adams: Hallelujah Junction: 3rd Movement   
928453   Shostakovich: Concerto 

In [ ]:
recommendations[['track_name', 'artist_name', 'final_score', 'similarity_score', 'weather_score', 'genre']]

,track_name,artist_name,final_score,similarity_score,weather_score,genre
883452,Concerto for Harpsichord and Orchestra: Moveme...,Philip Glass,0.786199,0.572397,1.0,ambient
800270,Bluebeard,Would-Be-Goods,0.770975,0.541951,1.0,club
882683,Death Is a Disease,Clint Mansell,0.764601,0.529202,1.0,ambient
47862,What Was Good Enough for You,Jeremy Jordan,0.763799,0.527598,1.0,show-tunes
928612,Salvation Mountain,Michael Brook,0.757858,0.515717,1.0,ambient
921528,Till You Lay Down Your Heavy Load,Eilen Jewell,0.756145,0.512289,1.0,singer-songwriter
1160558,Son of Placenta Previa,Cliff Martinez,0.753327,0.506653,1.0,ambient
795515,Adams: Hallelujah Junction: 3rd Movement,John Adams,0.751460,0.502921,1.0,ambient
928453,"Shostakovich: Concerto for Piano, Trumpet and ...",Dmitri Shostakovich,0.751346,0.502692,1.0,ambient
1100703,"Watchman, Tell Us of the Night",Giovanni,0.751029,0.502058,1.0,opera


In [ ]:
pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 7.0 MB/s eta 0:00:00


## Failed attempt to create playlist using api
 because ```playlist = sp.user_playlist_create(user=user_id, name=playlist_name, public=public, description=description)``` takes too long to process. That's why recommendations for listeners were added manually.

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Spotify API credentials
SPOTIPY_CLIENT_ID = '0143aa932e9a441c8e2bfcfcd99b2c58'
SPOTIPY_CLIENT_SECRET = 'e626f3c6e7f64ebe893b60b83d292726'
SPOTIPY_REDIRECT_URI = 'http://localhost:1111/callback'

# Authenticate with Spotify
scope = "playlist-modify-public playlist-modify-private"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET,
    redirect_uri=SPOTIPY_REDIRECT_URI,
    scope=scope
))

# Manually enter the user ID here
user_id = '31iyu4teq2efgg75iio52drwpw6e'

# Function to create a Spotify playlist and add tracks
def create_spotify_playlist(recommendations, playlist_name="Weather-Based Recommendations", public=True, description="A playlist based on weather and mood recommendations"):
    # Create a new playlist
    playlist = sp.user_playlist_create(user=user_id, name=playlist_name, public=public, description=description)
    playlist_id = playlist['id']

    # Search and add songs to the playlist
    track_uris = []
    for _, row in recommendations.iterrows():
        query = f"{row['track_name']} {row['artist_name']}"
        results = sp.search(q=query, type="track", limit=1)
        if results['tracks']['items']:
            track_uri = results['tracks']['items'][0]['uri']
            track_uris.append(track_uri)

    # Add tracks to the playlist
    if track_uris:
        sp.playlist_add_items(playlist_id, track_uris)
        print(f"Added {len(track_uris)} tracks to the playlist '{playlist_name}'.")
    else:
        print("No tracks were found on Spotify to add to the playlist.")

# Main function (partial for clarity)
if __name__ == "__main__":
    # Ensure the 'recommendations' DataFrame is created before this step
    # Save recommendations to Spotify
    create_spotify_playlist(recommendations)


KeyboardInterrupt: 